In [7]:
import gurobipy as gp
from gurobipy import GRB

R = ['R1', 'R2'] # assume 3 retailers
N = ['N1', 'N2'] # assume 3 nodes, suppliers ship to warehouses who ship to retailers
F = ['F1'] # assume 2 suppliers
p = 25  # Unit product price
c = 10  # Unit production cost
c_st = {('N1', 'N2'): 5}  # Transportation costs between nodes
b_r = {'R1': 100, 'R2': 150}  # Random demand for the product at each retailer
m_s = {'F1': 200}  # Maximum supply for each supplier
m_td = {'N1': 300, 'N2': 400}  # Maximum transshipment quantity

model = gp.Model("SupplierProfitOptimization")

# Decision variables
x_st = model.addVars(N, N, name="x_st", vtype=GRB.CONTINUOUS, lb=0)  # Quantity shipped through each node
s_f = model.addVars(F, name="s_f", vtype=GRB.CONTINUOUS, lb=0)  # Production quantity of suppliers

# Set objective: Maximize profit
model.setObjective(
    gp.quicksum(p * b_r[r] for r in R) - #total profit
    gp.quicksum(c_st[s, t] * x_st[s, t] for s in N for t in N if (s, t) in c_st) - #transportation costs
    gp.quicksum(c * s_f[f] for f in F), #production costs
    GRB.MAXIMIZE
)

# Constraints
# Supply-Demand Balance
model.addConstr(
    gp.quicksum(s_f[f] for f in F) >= gp.quicksum(b_r[r] for r in R), "SupplyDemandBalance"
)

# Flow Conservation
for f in F:
    model.addConstr(
        gp.quicksum(x_st[f, t] for t in N if (f, t) in c_st) == s_f[f], f"FlowConservation_{f}"
    )

# Supply Capacity
for f in F:
    model.addConstr(
        s_f[f] <= m_s[f], f"SupplyCapacity_{f}"
    )

# Demand Fulfillment
for r in R:
    model.addConstr(
        gp.quicksum(x_st[s, r] for s in N if (s, r) in c_st) == b_r[r], f"DemandFulfillment_{r}"
    )

# Transshipment Conservation
for d in N:
    model.addConstr(
        gp.quicksum(x_st[s, d] for s in N if (s, d) in c_st) == gp.quicksum(x_st[d, t] for t in N if (d, t) in c_st))
    
    


model.optimize()

# Check if the model found an optimal solution
if model.status == GRB.OPTIMAL:
    # Print the optimal objective value
    print(f"Optimal Profit: {model.objVal}")

    # Print the optimal values of the decision variables
    print("\nOptimal production quantities (s_f):")
    for f in F:
        print(f"  {f}: {s_f[f].x}")

    print("\nOptimal shipment quantities (x_st):")
    for s, t in c_st.keys():
        print(f"  From {s} to {t}: {x_st[s, t].x}")

else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 7 rows, 5 columns and 5 nonzeros
Model fingerprint: 0xe128b514
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model
No optimal solution found.


In [ ]:
import gurobipy as gp
from gurobipy import GRB

R = ['R1', 'R2', 'R3'] # assume 3 retailers
N = ['N1', 'N2', 'N3'] # assume 3 nodes, suppliers ship to warehouses who ship to retailers
F = ['F1', 'F2'] # assume 2 suppliers
p = 25  # Unit product price
c = 10  # Unit production cost
c_st = {('N1', 'N2'): 5, ('N2', 'N3'): 5}  # Transportation costs between nodes
b_r = {'R1': 100, 'R2': 150, 'R3': 200}  # Random demand for the product at each retailer
m_s = {'F1': 200, 'F2': 250}  # Maximum supply for each supplier
m_td = {'N1': 300, 'N2': 400, 'N3': 350}  # Maximum transshipment quantity

model = gp.Model("SupplierProfitOptimization")

# Decision variables
x_st = model.addVars(N, N, name="x_st", vtype=GRB.CONTINUOUS, lb=0)  # Quantity shipped through each node
s_f = model.addVars(F, name="s_j", vtype=GRB.CONTINUOUS, lb=0)  # Production quantity of suppliers

# Set objective: Maximize profit
model.setObjective(
    gp.quicksum(p * b_r[r] for r in R) - #total profit
    gp.quicksum(c_st[s, t] * x_st[s, t] for s in N for t in N if (s, t) in c_st) - #transportation costs
    gp.quicksum(c * s_f[f] for f in F), #production costs
    GRB.MAXIMIZE
)

# Constraints
# Supply-Demand Balance
model.addConstr(
    gp.quicksum(s_f[f] for f in F) >= gp.quicksum(b_r[r] for r in R), "SupplyDemandBalance"
)

# Flow Conservation
for f in F:
    model.addConstr(
        gp.quicksum(x_st[f, t] for t in N if (f, t) in c_st) == s_f[f], f"FlowConservation_{f}"
    )

# Supply Capacity
for f in F:
    model.addConstr(
        s_f[f] <= m_s[f], f"SupplyCapacity_{f}"
    )

# Demand Fulfillment
for r in R:
    model.addConstr(
        gp.quicksum(x_st[s, r] for s in N if (s, r) in c_st) == b_r[r], f"DemandFulfillment_{r}"
    )

# Transshipment Conservation
for d in N:
    model.addConstr(
        gp.quicksum(x_st[s, d] for s in N if (s, d) in c_st) == gp.quicksum(x_st[d, t] for t in N if (d, t) in c_st))
    
    


model.optimize()

# Check if the model found an optimal solution
if model.status == GRB.OPTIMAL:
    # Print the optimal objective value
    print(f"Optimal Profit: {model.objVal}")

    # Print the optimal values of the decision variables
    print("\nOptimal production quantities (s_f):")
    for f in F:
        print(f"  {f}: {s_f[f].x}")

    print("\nOptimal shipment quantities (x_st):")
    for s, t in c_st.keys():
        print(f"  From {s} to {t}: {x_st[s, t].x}")

else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 11 rows, 11 columns and 10 nonzeros
Model fingerprint: 0x0af3a35d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 4e+02]
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model
No optimal solution found.
